In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
# data loader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("resume.txt", encoding = 'UTF-8')

In [ ]:
# pdf loader
# from langchain_community.document_loaders import PyPDFLoader

# file_path = ".pdf"
# loader = PyPDFLoader(file_path)

# docs = loader.load()

# print(len(docs))

In [ ]:
docs = loader.load()
docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

In [ ]:
all_splits

In [ ]:
# indexing
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [ ]:
# retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Who is navneet?")

len(retrieved_docs)

In [ ]:
retrieved_docs

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.if question is not related to context then dont answer them."
    "your creater is navneet"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "make two mcq question to check navneet's python skills level"})
print(response["answer"])

In [ ]:
response

In [ ]:
response = rag_chain.invoke({"input": "what is use of tensorflow"})
print(response["answer"])